In [1]:
from docx import Document
from pathlib import Path
import pandas as pd
import re

In [2]:
Less_Than_Twenty = {
    0: '',
    1: 'One',
    2: 'Two',
    3: 'Three',
    4: 'Four',
    5: 'Five',
    6: 'Six',
    7: 'Seven',
    8: 'Eight',
    9: 'Nine',
    10: 'Ten',
    11: 'Eleven',
    12: 'Twelve',
    13: 'Thirteen',
    14: 'Fourteen',
    15: 'Fifteen',
    16: 'Sixteen',
    17: 'Seventeen',
    18: 'Eighteen',
    19: 'Nineteen'
}

In [3]:
Less_Than_Hundred = {
    2: 'Twenty',
    3: 'Thirty',
    4: 'Forty',
    5: 'Fifty',
    6: 'Sixty',
    7: 'Seventy',
    8: 'Eighty',
    9: 'Ninety'
}

In [4]:
More_Than_Thousand = {
    1: 'Thousand',
    2: 'Million',
    3: 'Biliion',
    4: 'Trillion'
}

In [5]:
def num_to_word(num):
    if type(num) == float:
        decimal = round(num - int(num), 2)
        if decimal:
            text = ' '.join([num_to_word(int(num)), 'and Cents', num_to_word(int(decimal * 100)), 'Only'])
        else:
            text = ' '.join([num_to_word(int(num)), 'Only'])
        text = text.replace('  ', ' ')
        return text
    num_digit = len(str(num))
    power = (num_digit - 1) // 3
    if num < 20:
        text = Less_Than_Twenty[num]
        return text
    elif num < 100:
        text = ' '.join([Less_Than_Hundred[num // 10], num_to_word(num % 10)])
        return text
    elif num < 1000:
        text = ' '.join([num_to_word(num // 100), 'Hundred', num_to_word(num % 100)])
        return text
    elif power < 5:
        text = ' '.join([num_to_word(num // 1000 ** power), More_Than_Thousand[power], num_to_word(num % 1000 ** power)])
        return text

In [6]:
def get_table(file_path):
    df = pd.read_excel(file_path)
    
    amounts = ['Amount' + str(i + 1) for i in range(8)]
    for index, amount in enumerate(amounts):
        if index == 0:
            df['Total'] = df[amount]
        elif not df[amount].isna().sum():
            df['Total'] = df['Total'] + df[amount]
    df['Ringgit'] = df['Total'].apply(num_to_word)
    
    df['Date'] = df['Date'].dt.strftime('%d/%m/%Y')
    for amount in amounts:
        if not df[amount].isna().sum():
            df[amount] = df[amount].apply(lambda row: f'{row:,.2f}')
    df['Total'] = df['Total'].apply(lambda row: f'{row:,.2f}')
    df = df.fillna('')
    
    table = df.to_dict('records')
    return table

In [7]:
input_path = Path.cwd() / 'input'
output_path = Path.cwd() / 'output'
template_path = Path.cwd() / 'template'

In [8]:
file_list = list(input_path.glob('*.xlsx'))
file_path = file_list[0]
file_path

WindowsPath('c:/Users/syenl/Documents/Python-Tools/work/voucher/input/payment-voucher.xlsx')

In [9]:
template = list(template_path.glob(f'{file_path.stem}.docx'))[0]
template

WindowsPath('c:/Users/syenl/Documents/Python-Tools/work/voucher/template/payment-voucher.docx')

In [10]:
table = get_table(file_path)

In [11]:
def create_voucher(template, data, target_path):
    doc = Document(template)
    table = doc.tables[0]
    for row in table.rows:
        for item in row.cells:
            pattern = r'\$\{(.*?)\}'
            result = re.findall(pattern, item.text)
            if result:
                key = result[0]
                value = data[key]
                for index, run in enumerate(item.paragraphs[0].runs):
                    if index == 0:
                        run.text = str(value)
                    else:
                        run.text = ''
    doc.save(target_path)

In [12]:
for index, data in enumerate(table):
    target_path = output_path / f'{file_path.stem}-{index + 1}.docx'
    create_voucher(template, data, target_path)